In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd
from datamatch import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)

from lib.columns import rearrange_complaint_columns
from lib.explore import print_df
from clean.baton_rouge_pd_cprr import clean_18
from clean.baton_rouge_csd_pprr import clean_17 as clean_csd_17, clean_19 as clean_csd_19
from clean.baton_rouge_pd_pprr import clean as clean_pprr

In [2]:
df17, df19 = clean_csd_17(), clean_csd_19()
df = clean_pprr()
df

,rank_desc,badge_no,last_name,first_name,middle_initial,data_production_year,agency,uid
0,cpl,291,abadie,jonathan,m,2021,Baton Rouge PD,572b5ea334efc3ff94da634ae323b930
1,cpl,457,acree,jason,k,2021,Baton Rouge PD,a02b3a98a05c9b8fedf038032d0b094e
2,off,1211,adams,aimee,,2021,Baton Rouge PD,b6f1552b3c0ef8c8b17562704971dbf0
3,off,1201,adams,dominique,,2021,Baton Rouge PD,48a35ff775efcb6cb53dbea5127a38fc
4,off,1107,adams-walker,ebony,,2021,Baton Rouge PD,2b3884808972f05549fe89ed93cdfff7
...,...,...,...,...,...,...,...,...
605,sgt,170,woodard,rodrick,e,2021,Baton Rouge PD,6d82e73c3d401a0424eef198d7ace669
606,cpl,275,woodring,steven,z,2021,Baton Rouge PD,86ed1bb573d54cc9791cd7adcccb68d5
607,cpl,430,wright,christopher,j,2021,Baton Rouge PD,464b671eff4b54c1765d114ea064d8ac
608,sgt,176,young jr,donald,p,2021,Baton Rouge PD,6503fb0a070a37136a1e291cf7430915


In [11]:
len(df.uid.unique())

610

In [9]:
len(df17.uid.unique())

1660

In [10]:
len(df19.uid.unique())

1057